In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

from copy import deepcopy
import itertools
import pickle

import pandas as pd
from scipy.io import savemat, loadmat
import seaborn as sns
from sklearn.metrics import (log_loss, mean_squared_error, roc_curve, auc,
                             precision_recall_fscore_support, confusion_matrix)
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from tqdm import tqdm, trange, tqdm_notebook as tqdmn

from BEKML import BEMKL, plot_distplot
from utils import poly_kernel, gauss_kernel, scoring, plot_kernel_importances, plot_compare_models

sns.set(style='ticks', context='talk')
np.set_printoptions(precision=4, linewidth=100)

Populating the interactive namespace from numpy and matplotlib


/home/mtambos/anaconda/envs/pml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('data/wdbc.csv', names=list(range(32)), index_col=0)
display(data.head())
data.describe()

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [3]:
X = data.iloc[:, 1:].copy()
y = data.iloc[:, 0].copy()
N, D = X.shape
y.loc[y == 'B'] = -1
y.loc[y == 'M'] = 1
y = y.astype(int)
display(X.shape, y.shape, y.mean())

(569, 30)

(569,)

-0.2548330404217926

In [4]:
proc_data = data.copy()
proc_data.iloc[:, 0] = y
proc_data = proc_data.loc[:, list(proc_data.columns[1:]) + [proc_data.columns[0]]]
proc_data.to_csv('data/proc_wdbc.csv')
proc_data.head()

,2,3,4,5,6,7,8,9,10,11,...,23,24,25,26,27,28,29,30,31,1
0,,,,,,,,,,,,,,,,,,,,,
842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [5]:
rbf_init, rbf_end = -3, 7
ply_init, ply_end = 1, 4

kernel_attrs = [('rbf', 'all', i) for i in range(rbf_init, rbf_end)]
kernels = [lambda A, B: gauss_kernel(A, B, 2**i)
           for i in range(rbf_init, rbf_end)]

kernel_attrs += [('poly', 'all', i) for i in range(ply_init, ply_end)]
kernels += [lambda A, B: poly_kernel(A, B, 1, i)
            for i in range(ply_init, ply_end)]

kernel_attrs += [('rbf', j, i) for i in range(rbf_init, rbf_end)
                 for j in range(D)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i)
            for i in range(rbf_init, rbf_end) for j in range(D)]

kernel_attrs += [('poly', j, i) for i in range(ply_init, ply_end)
                 for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 1, i)
            for i in range(ply_init, ply_end) for j in range(D)]
len(kernels), len(kernel_attrs)

(403, 403)

In [6]:
X_train, X_test, y_train, y_test =\
        train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [7]:
normalizer = Normalizer()
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.fit_transform(X_test)

# Baseline Model

In [8]:
max_iter = 200
base_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                   hyp_gamma_alpha=1, hyp_gamma_beta=1,
                   hyp_omega_alpha=1, hyp_omega_beta=1,
                   e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                   filter_kernels=False, filter_sv=False, verbose=False,
                   max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
base_model = make_pipeline(Normalizer(), base_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
base_cv_results = cross_validate(base_model, X, y, cv=folds, scoring=scoring)
base_stats = deepcopy(scoring.stats)
base_cv_results, base_stats

0 - Kernels: 13/403 (0.03225806451612903). SV: 379/426 (0.8896713615023474). Mean e: 0.0163. Median e: 0.0088. Std e: 0.0819. 
1 - Kernels: 13/403 (0.03225806451612903). SV: 420/427 (0.9836065573770492). Mean e: 0.0150. Median e: 0.0077. Std e: 0.0790. 
2 - Kernels: 13/403 (0.03225806451612903). SV: 377/427 (0.882903981264637). Mean e: 0.0125. Median e: 0.0057. Std e: 0.0738. 
3 - Kernels: 13/403 (0.03225806451612903). SV: 389/427 (0.9110070257611241). Mean e: 0.0136. Median e: 0.0066. Std e: 0.0772. 
4 - Kernels: 13/403 (0.03225806451612903). SV: 406/426 (0.9530516431924883). Mean e: 0.0144. Median e: 0.0071. Std e: 0.0798. 
5 - Kernels: 13/403 (0.03225806451612903). SV: 361/427 (0.8454332552693209). Mean e: 0.0145. Median e: 0.0076. Std e: 0.0750. 
6 - Kernels: 13/403 (0.03225806451612903). SV: 416/427 (0.9742388758782201). Mean e: 0.0127. Median e: 0.0049. Std e: 0.0847. 
7 - Kernels: 13/403 (0.03225806451612903). SV: 374/427 (0.8758782201405152). Mean e: 0.0145. Median e: 0.0076. S

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([23.5434, 24.3743, 25.6749, 22.9158, 22.8114, 23.1622, 23.5647, 23.0302, 23.1108, 22.9726,
         23.8407, 23.2623, 23.1117, 23.8848, 23.6176, 23.0503, 22.8715, 23.6353, 26.4198, 22.7334]),
  'score_time': array([4.6688, 4.3916, 4.2891, 4.2866, 4.2724, 4.2657, 4.3258, 4.2519, 4.4902, 4.3054, 4.7011,
         4.3482, 4.3192, 4.3132, 4.3267, 4.3079, 4.2729, 4.4355, 4.2855, 4.3507]),
  'test_score': array([0.9091, 0.9014, 0.9225, 0.9296, 0.9371, 0.9155, 0.9155, 0.9155, 0.9161, 0.8944, 0.9014,
         0.9437, 0.951 , 0.9085, 0.9014, 0.9085, 0.9021, 0.8944, 0.9366, 0.9437]),
  'train_score': array([0.9272, 0.9274, 0.9157, 0.918 , 0.9178, 0.9227, 0.9274, 0.9227, 0.9225, 0.9251, 0.9274,
         0.904 , 0.9085, 0.9251, 0.9274, 0.9204, 0.9296, 0.9274, 0.9157, 0.9133])},
 [{'elapsed_time': 21.027663230895996,
   'nr_kernels_used': 13,
   'nr_sv_used': 379,
   'total_kernels': 403,
   'total_sv': 426},
  {'elapsed_time': 21.921558380126953,
   'nr_kernels_used': 13,
   'nr

In [9]:
base_times = np.array([s['elapsed_time'] for s in base_stats])
base_kernels = np.array([s['nr_kernels_used'] for s in base_stats])
base_sv = np.array([s['nr_sv_used'] for s in base_stats])
print(
    f"Score: {base_cv_results['test_score'].mean()} +- {base_cv_results['test_score'].std()}\n"
    f"Time: {base_times.mean()} +- {base_times.std()}\n"
    f"Kernels: {base_kernels.mean()} +- {base_kernels.std()}\n"
    f"SVs: {base_sv.mean()} +- {base_sv.std()}"
)

Score: 0.9173889490790899 +- 0.016950870305825923
Time: 21.112578201293946 +- 0.9248643330677546
Kernels: 13.0 +- 0.0
SVs: 380.4 +- 24.607722365143832


# Kernel-sparse model

In [10]:
max_iter = 200
ksparse_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1e-11, hyp_omega_beta=1e9,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ksparse_pipeline = make_pipeline(Normalizer(), ksparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ksparse_cv_results = cross_validate(ksparse_pipeline, X, y, cv=folds, scoring=scoring)
ksparse_stats = deepcopy(scoring.stats)
ksparse_cv_results, ksparse_stats

0 - Kernels: 3/403 (0.007444168734491315). SV: 375/426 (0.8802816901408451). Mean e: 0.0068. Median e: 0.0001. Std e: 0.0774. 
1 - Kernels: 3/403 (0.007444168734491315). SV: 344/427 (0.8056206088992974). Mean e: 0.0064. Median e: 0.0001. Std e: 0.0730. 
2 - Kernels: 3/403 (0.007444168734491315). SV: 294/427 (0.6885245901639344). Mean e: 0.0069. Median e: 0.0001. Std e: 0.0793. 
3 - Kernels: 3/403 (0.007444168734491315). SV: 340/427 (0.7962529274004684). Mean e: 0.0073. Median e: 0.0001. Std e: 0.0837. 
4 - Kernels: 3/403 (0.007444168734491315). SV: 375/426 (0.8802816901408451). Mean e: 0.0071. Median e: 0.0001. Std e: 0.0811. 
5 - Kernels: 3/403 (0.007444168734491315). SV: 352/427 (0.8243559718969555). Mean e: 0.0069. Median e: 0.0001. Std e: 0.0792. 
6 - Kernels: 3/403 (0.007444168734491315). SV: 329/427 (0.7704918032786885). Mean e: 0.0065. Median e: 0.0001. Std e: 0.0742. 
7 - Kernels: 3/403 (0.007444168734491315). SV: 414/427 (0.9695550351288056). Mean e: 0.0069. Median e: 0.0001. 

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([31.8894, 27.2667, 23.2289, 22.9908, 22.9379, 23.9261, 23.3843, 23.48  , 22.7919, 22.8237,
         23.0361, 22.7708, 24.0527, 23.2313, 22.8116, 22.8613, 23.1158, 22.9194, 22.7761, 23.2873]),
  'score_time': array([4.7327, 4.248 , 4.4103, 4.4189, 4.3075, 4.3359, 4.3122, 4.3113, 4.2577, 4.3865, 4.2484,
         4.3767, 4.4139, 4.2854, 4.3969, 4.2681, 4.2867, 4.2564, 4.2725, 4.2994]),
  'test_score': array([0.9091, 0.9296, 0.9437, 0.9085, 0.9091, 0.9155, 0.9437, 0.9014, 0.9021, 0.9296, 0.9085,
         0.9437, 0.9231, 0.9437, 0.9296, 0.9014, 0.8951, 0.9507, 0.9014, 0.9225]),
  'train_score': array([0.9249, 0.918 , 0.9087, 0.9297, 0.9225, 0.9251, 0.911 , 0.9321, 0.9296, 0.9204, 0.9251,
         0.9157, 0.9202, 0.9157, 0.918 , 0.9274, 0.9272, 0.911 , 0.9274, 0.9227])},
 [{'elapsed_time': 29.457967281341553,
   'nr_kernels_used': 3,
   'nr_sv_used': 375,
   'total_kernels': 403,
   'total_sv': 426},
  {'elapsed_time': 24.782368183135986,
   'nr_kernels_used': 3,
   'nr_s

In [11]:
ksparse_times = np.array([s['elapsed_time'] for s in ksparse_stats])
ksparse_kernels = np.array([s['nr_kernels_used'] for s in ksparse_stats])
ksparse_sv = np.array([s['nr_sv_used'] for s in ksparse_stats])
print(
    f"Score: {ksparse_cv_results['test_score'].mean()} +- {ksparse_cv_results['test_score'].std()}\n"
    f"Time: {ksparse_times.mean()} +- {ksparse_times.std()}\n"
    f"Kernels: {ksparse_kernels.mean()} +- {ksparse_kernels.std()}\n"
    f"SVs: {ksparse_sv.mean()} +- {ksparse_sv.std()}"
)

Score: 0.9205850487540627 +- 0.01727545744937103
Time: 21.311969101428986 +- 2.1040136595700645
Kernels: 3.0 +- 0.0
SVs: 351.2 +- 33.49865668948532


# SV-sparse model

In [12]:
max_iter = 200
ssparse_model = BEMKL(kernels=kernels,
                      hyp_lambda_alpha=1e-11, hyp_lambda_beta=1e9,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1, hyp_omega_beta=1,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ssparse_pipeline = make_pipeline(Normalizer(), ssparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ssparse_cv_results = cross_validate(ssparse_pipeline, X, y, cv=folds, scoring=scoring)
ssparse_stats = deepcopy(scoring.stats)
ssparse_cv_results, ssparse_stats

0 - Kernels: 13/403 (0.03225806451612903). SV: 323/426 (0.7582159624413145). Mean e: 0.0133. Median e: 0.0064. Std e: 0.0750. 
1 - Kernels: 13/403 (0.03225806451612903). SV: 348/427 (0.8149882903981265). Mean e: 0.0151. Median e: 0.0080. Std e: 0.0773. 
2 - Kernels: 13/403 (0.03225806451612903). SV: 333/427 (0.7798594847775175). Mean e: 0.0139. Median e: 0.0063. Std e: 0.0833. 
3 - Kernels: 13/403 (0.03225806451612903). SV: 332/427 (0.7775175644028103). Mean e: 0.0173. Median e: 0.0098. Std e: 0.0815. 
4 - Kernels: 13/403 (0.03225806451612903). SV: 342/426 (0.8028169014084507). Mean e: 0.0142. Median e: 0.0067. Std e: 0.0813. 
5 - Kernels: 13/403 (0.03225806451612903). SV: 371/427 (0.8688524590163934). Mean e: 0.0147. Median e: 0.0076. Std e: 0.0780. 
6 - Kernels: 13/403 (0.03225806451612903). SV: 361/427 (0.8454332552693209). Mean e: 0.0147. Median e: 0.0072. Std e: 0.0813. 
7 - Kernels: 13/403 (0.03225806451612903). SV: 308/427 (0.7213114754098361). Mean e: 0.0152. Median e: 0.0080. 

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([22.9817, 22.7917, 22.9075, 22.82  , 22.8179, 22.9772, 23.0265, 23.0023, 22.8389, 22.9372,
         23.0317, 22.8454, 22.9831, 22.8465, 22.7964, 23.0949, 22.8364, 22.8416, 23.0125, 23.1468]),
  'score_time': array([4.2466, 4.246 , 4.305 , 4.2507, 4.2806, 4.4149, 4.3151, 4.2562, 4.2807, 4.2554, 4.2546,
         4.26  , 4.2593, 4.2781, 4.2377, 4.2882, 4.3017, 4.2489, 4.252 , 4.329 ]),
  'test_score': array([0.9301, 0.9366, 0.9155, 0.9014, 0.9231, 0.8944, 0.9155, 0.9507, 0.9231, 0.8944, 0.9366,
         0.9225, 0.9161, 0.9155, 0.9296, 0.9085, 0.9161, 0.9437, 0.8803, 0.9296]),
  'train_score': array([0.9155, 0.9204, 0.9251, 0.9297, 0.9178, 0.9321, 0.9274, 0.9087, 0.9178, 0.9321, 0.9133,
         0.9251, 0.9202, 0.9227, 0.9157, 0.9274, 0.9225, 0.911 , 0.9344, 0.9204])},
 [{'elapsed_time': 20.53350043296814,
   'nr_kernels_used': 13,
   'nr_sv_used': 323,
   'total_kernels': 403,
   'total_sv': 426},
  {'elapsed_time': 20.345441341400146,
   'nr_kernels_used': 13,
   'nr_

In [13]:
ssparse_times = np.array([s['elapsed_time'] for s in ssparse_stats])
ssparse_kernels = np.array([s['nr_kernels_used'] for s in ssparse_stats])
ssparse_sv = np.array([s['nr_sv_used'] for s in ssparse_stats])
print(
    f"Score: {ssparse_cv_results['test_score'].mean()} +- {ssparse_cv_results['test_score'].std()}\n"
    f"Time: {ssparse_times.mean()} +- {ssparse_times.std()}\n"
    f"Kernels: {ssparse_kernels.mean()} +- {ssparse_kernels.std()}\n"
    f"SVs: {ssparse_sv.mean()} +- {ssparse_sv.std()}"
)

Score: 0.9191519747857775 +- 0.017048868513916685
Time: 20.466110670566557 +- 0.08944214821566429
Kernels: 13.0 +- 0.0
SVs: 338.3 +- 26.771440006096046


In [14]:
import json
with open('wdbc_results.json', 'w') as fp:
    json.dump(
        {
            'ksparse': {
                'scores': list(ksparse_cv_results['test_score']),
                'times': list(ksparse_times),
                'kernels': [int(k) for k in ksparse_kernels],
                'svs': [int(s) for s in ksparse_sv],
            },
            'ssparse': {
                'scores': list(ssparse_cv_results['test_score']),
                'times': list(ssparse_times),
                'kernels': [int(k) for k in ssparse_kernels],
                'svs': [int(s) for s in ssparse_sv],
            },
            'base': {
                'scores': list(base_cv_results['test_score']),
                'times': list(base_times),
                'kernels': [int(k) for k in base_kernels],
                'svs': [int(s) for s in base_sv],
            },
            'total_kernels': len(kernels),
            'total_sv': len(X_train),
        },
        fp,
        indent=4,
        sort_keys=True
    )